In [1]:
# importing general packages

# importing the necessary packages:

import numpy as np
import sys
import os
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'

plt.rc('font', family='serif', size=18)
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['ytick.major.size'] = 5

%matplotlib inline

# importing ALP packages

sys.path.append('/Users/milena/Desktop/2nd_yr_project/ALPs/ALPs_analysis/') # path to calc_alp_signal script
from calc_alp_signal import ALPSNSignal

# importing additional packages

from scipy import integrate

### Sanity check:
Idea: take the lowest normalization (5.04e-52 cm^-2); integrate over the ALP spectrum over time & energy, to get fluence. How does it compare to the fluences reported in the catalog?

In [4]:
EMeV = np.linspace(20.,300,100) # [MeV]
ts = np.linspace(0.1,25.,250)
ee, tt = np.meshgrid(EMeV,ts, indexing = 'ij')
alp_sn10 = ALPSNSignal(Mprog = 10.) # in 10^52
dndedt_alp10 = alp_sn10(EMeV, ts, g10=0.1) # Flux, [10^52 ph/(MeV s)]

SED10 = integrate.simps(dndedt_alp10, ts)

In [5]:
# lowest normalization for which we are sensitive to ALPs, medium background, P=0.1
norm = 3.68 # [cm^-2], in  10^-52
fluence = norm*integrate.simps(SED10, EMeV)/25. #averaging over time elapsed
print("The fluence of the ALP signal corresponding to normalization 6.3e-52 is %.3f cts/ s cm^2." %fluence)

The fluence of the ALP signal corresponding to normalization 6.3e-52 is 0.039 cts/ s cm^2.


This is the value averaged over 25 seconds of the ALP emission.

#### GRB 080825 C
8 sigma detection in LLE: GRB 080825C: https://arxiv.org/pdf/0910.4192.pdf
LLE analysis spectral fit given below:

In [5]:
# The lowest reported detection in the catalog in LLE
# 8 sigma detection: GRB 080825C: https://arxiv.org/pdf/0910.4192.pdf
# LLE Band function spectral parameters:
norm = 138e-3
alpha = -0.52
beta = -2.37
tem = 210 # [keV]

def get_parVals():
    pars = [alpha, beta, tem, norm]    
    return pars

In [6]:
# Defining the Band function: https://ui.adsabs.harvard.edu/abs/1993ApJ...413..281B/abstract
# in XSPEC: https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/node181.html

def band(energy):
    '''
    BAND(energy, alpha, beta, tem, norm) 
        
    PARAMETERS:
    ----------
    energy: float, energy element from an array of energies over which 
            the function is integrated over. 
    alpha:  float, alpha (low-energy index) parameter value.  
    beta:   float, beta (high-energy index) parameter value.     
    tem: float, characteristic energy parameter value.  
    norm:   float, normalization (aka amplitude) parameter value.  
    ''' 
    alpha, beta, tem, norm = get_parVals()

    a = float(alpha)
    b = float(beta)
    t = float(tem)
    N = float(norm)
    eng  = energy
    
    if eng < ( (a-b) * t ):
        return  N * (((eng/100.0)**a) * (np.exp(-eng/t)))
    else:
        return  N * (((((a-b)*t)/100.0)**(a-b)) * (np.exp(b-a))*((eng/100.0)**b))

def eband(energy):
    eng  = energy
    return eng * band(eng)

In [12]:
keVtoerg    = 1.60217657E-9 # conversion factor from keV to erg
emin        = 20000. 
emax        = 300000.

In [13]:
Flux_Ph = integrate.quad(band, emin, emax, limit=100)[0]
Flux_En = integrate.quad(eband, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t cts s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  0.013394103 	 cts s^-1 cm^-2
Energy Flux:  1.030942536e-06 	 ergs s^-1 cm^-2

